アノテーションした点をBBOXに変換する

In [5]:
import os
import os.path as osp
from glob import glob
from util import load_json, save_json
import shutil
import numpy as np
from tqdm import tqdm

In [6]:
data_name = 'tennis'
# data_name = 'ball'
# length = 10
length = 20

In [7]:
base_dir = '/workspace/CenterTrack/'
data_dir = os.path.join(base_dir, 'input/{}/'.format(data_name))
save_dir = os.path.join(base_dir, 'output/{}/bbox/'.format(data_name))

In [8]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [9]:
games = listdirs(data_dir, 'game')
for game in tqdm(games):
    game_dir = osp.join(data_dir, game)
    clips = listdirs(game_dir, 'Clip')    
    for clip in clips:
        clip_dir = osp.join(game_dir, clip)
        file_names = listdirs(clip_dir)
        label_names = [label for label in file_names if osp.splitext(label)[1]=='.json']
        image_names = [image for image in file_names if osp.splitext(image)[1]=='.jpg']
        
        label_names.sort()
        image_names.sort()
        
        if len(label_names) != len(image_names):
            print(game, '-', clip)
            print('label_names {} and image_names {} are not different'.format(len(label_names), len(image_names)))
            continue
        
        for image_name, label_name in zip(image_names, label_names):
            assert osp.splitext(label_name)[0] == osp.splitext(image_name)[0]
            
            ##### path #####
            frame_index, ext = osp.splitext(image_name)
            save_name = '{}-{}-{}'.format(game, clip, frame_index)
            label_path = osp.join(clip_dir, label_name)
            image_path = osp.join(clip_dir, image_name)
            
            image_save_path = osp.join(save_dir, game, clip, '{}.jpg'.format(save_name))
            label_save_path = osp.join(save_dir, game, clip, '{}.json'.format(save_name))
            
            os.makedirs(osp.dirname(image_save_path), exist_ok=True)
            
            ##### image #####
            shutil.copy(image_path, image_save_path)
            
            ##### label #####
            label = load_json(label_path)
            label['imageData'] = None
            
            # bbox
            label_shapes = label['shapes']
            for label_num in range(len(label_shapes)):
                data = label_shapes[label_num]
                
                label_name = data['label']
                if label_name == 'person':
                    xmin, ymin = data['points'][0]
                    xmax, ymax = data['points'][1]
                    
                    xmin = float(np.maximum(xmin, 0))
                    ymin = float(np.maximum(ymin, 0))
                    xmax = float(np.minimum(xmax, label['imageWidth']))
                    ymax = float(np.minimum(ymax, label['imageHeight']))
                else:
                    points = data['points']

                    x, y = points[0]

                    xmin = np.maximum(x - length / 2, 0)
                    ymin = np.maximum(y - length / 2, 0)
                    xmax = np.minimum(x + length / 2, label['imageWidth'])
                    ymax = np.minimum(y + length / 2, label['imageHeight'])

                bbox = [[xmin, ymin], [xmax, ymax]]

                width = xmax - xmin
                height = ymax - ymin

                if (width > 0) and (height > 0):
                    label['shapes'][label_num]['points'] = bbox
                    label['shapes'][label_num]['shape_type'] = 'rectangle'
                else:
                    print('no bbox')
                    del label['shapes'][label_num]
            
            # name
            label['imagePath'] = osp.basename(image_save_path)
            save_json(label, label_save_path)

100%|██████████| 10/10 [00:28<00:00,  2.89s/it]
